In [1]:
import keras.backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Dense, Dropout, Flatten
from sports1M_utils import preprocess_input, decode_predictions
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D

C:\Users\Othmane\Anaconda3\envs\theano\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
WEIGHTS_PATH = 'https://github.com/adamcasson/c3d/releases/download/v0.1/sports1M_weights_tf.h5'
C3D_MEAN_PATH = 'https://github.com/adamcasson/c3d/releases/download/v0.1/c3d_mean.npy'

In [3]:
def C3D(weights='sports1M'):
    """Instantiates a C3D Kerasl model
    
    Keyword arguments:
    weights -- weights to load into model. (default is sports1M)
    
    Returns:
    A Keras model.
    
    """
    
    if weights not in {'sports1M', None}:
        raise ValueError('weights should be either be sports1M or None')
    
    if K.image_data_format() == 'channels_last':
        shape = (16,112,112,3)
    else:
        shape = (3,16,112,112)
        
    model = Sequential()
    model.add(Conv3D(64, 3, activation='relu', padding='same', name='conv1', input_shape=shape))
    model.add(MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2), padding='same', name='pool1'))
    
    model.add(Conv3D(128, 3, activation='relu', padding='same', name='conv2'))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool2'))
    
    model.add(Conv3D(256, 3, activation='relu', padding='same', name='conv3a'))
    model.add(Conv3D(256, 3, activation='relu', padding='same', name='conv3b'))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool3'))
    
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv4a'))
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv4b'))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool4'))
    
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv5a'))
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv5b'))
    model.add(ZeroPadding3D(padding=(0,1,1)))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool5'))
    
    model.add(Flatten())
    
    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(0.5))
    model.add(Dense(487, activation='softmax', name='fc8'))

    if weights == 'sports1M':
        weights_path = get_file('sports1M_weights_tf.h5',
                                WEIGHTS_PATH,
                                cache_subdir='models',
                                md5_hash='b7a93b2f9156ccbebe3ca24b41fc5402')
        
        model.load_weights(weights_path)
    
    return model
    


# Access to videos 

In [7]:
import skvideo.io
from keras.models import Model
from sports1M_utils import preprocess_input, decode_predictions

def extract_features(video_sequence,model):
    

    
    L = sequence.shape[0]
    features = np.zeros((1,4096))
    nb_frames = 0
    
    for i in range(L//16):
        x = preprocess_input(sequence[16*i:16*(i+1)])
        features +=  model.predict(x)
        nb_frames = nb_frames + 1 
    
    features = features/max(1,nb_frames)

    return(features)


def get_sequences(video):
    L = video.shape[0]
    nbr_sequances = 32

    sequences_list = []

    for i in range(31):
        sequences_list.append(video[i*(L//nbr_sequances):(i+1)*(L//nbr_sequances)])

    sequences_list.append(video[31*L//nbr_sequances:])  
    return sequences_list
    

    

In [ ]:
import os
from os import listdir

base_model = C3D(weights='sports1M')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc6').output)

#DataSetPath = 'D:/workspace/MVA/ORCV/Final_Project/Data_Sets/UCF-Anomaly-Detection-Dataset/UCF_Crimes/Videos'

Normal_features_Path = "/Features/Normal"
Abnormal_features_Path = "/Features/Abnormal"

Normal_Videos_Path = '../Data_Sets/UCF-Anomaly-Detection-Dataset/UCF_Crimes/Videos/Normal'
Abnormal_Videos_Path = '../Data_Sets/UCF-Anomaly-Detection-Dataset/UCF_Crimes/Videos/Abnormal'


print("Abnormal featers extraction")
Abnormal_Videos= listdir(Abnormal_Videos_Path)
Abnormal_Videos.sort()



for directory in Abnormal_Videos:
    VideosPath = os.path.join(Abnormal_Videos_Path, directory)
    Videos_List = listdir(VideosPath)
    for Video_Path in Videos_List:
      
        #Get the name of the video 
        video_name = ''
        char = Video_Path[0]
        i = 0
        while (char!='.'):
            video_name += char
            i = i+1
            char = Video_Path[i]
        
            
        
        Video_Path = os.path.join(VideosPath, Video_Path)
        video = skvideo.io.vread(Video_Path)
        sequences_list = get_sequences(video)
        count = -1;
        for sequence in sequences_list:
            count += 1 
            if count == 0:
                features = extract_features(sequence, model)
            if count > 0:
                features = np.vstack((features,extract_features(sequence, model)))
        destination = os.path.join(Abnormal_features_Path, video_name + '.txt' )
        
        
        np.savetxt(destination, features, delimiter='  ')   
        print("Destination",destination)
        print("-----------------------------------")
        
    

print("Normal featers extraction")
Normal_Videos= listdir(Normal_Videos_Path)
Normal_Videos.sort()


mylist_features = []
for directory in Normal_Videos:
    VideosPath = os.path.join(Normal_Videos_Path, directory)
    Videos_List = listdir(VideosPath)
    for Video_Path in Videos_List:
      
        #Get the name of the video 
        video_name = ''
        char = Video_Path[0]
        i = 0
        while (char!='.'):
            video_name += char
            i = i+1
            char = Video_Path[i]
        
            
        
        Video_Path = os.path.join(VideosPath, Video_Path)
        video = skvideo.io.vread(Video_Path)
        sequences_list = get_sequences(video)
        count = -1;
        for sequence in sequences_list:
            count += 1 
            if count == 0:
                features = extract_features(sequence, model)
            if count > 0:
                features = np.vstack((features,extract_features(sequence, model)))
        destination = os.path.join(Abnormal_features_Path, video_name + '.txt' )
        
        mylist_features.append(features)
        np.savetxt(destination, features, delimiter='  ')   
        print("Destination",destination)
        print("-----------------------------------")

Abnormal featers extraction
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Abuse001_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Abuse002_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Abuse003_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Abuse004_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Abuse005_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Abuse0

Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Abuse050_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arrest001_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arrest002_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arrest003_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arrest004_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arrest005_x264.txt
-----------

Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arrest049_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arrest050_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arrest051_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arson001_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arson002_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arson003_x264.txt
-------------

Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arson050_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arson051_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arson052_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Arson053_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Assault001_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Assault002_x264.txt
------------

Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Assault047_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Assault048_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Assault049_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Assault050_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Assault051_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Assault052_x264.txt
----

Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Burglary043_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Burglary044_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Burglary045_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Burglary046_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Burglary047_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Burglary048_x264.tx

Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Burglary091_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Burglary092_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Burglary093_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Burglary094_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Burglary095_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Burglary096_x264.tx

Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Explosion040_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Explosion041_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Explosion042_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Explosion043_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Explosion044_x264.txt
-----------------------------------
Destination D:/workspace/MVA/ORCV/Final_Project/Waqas_Sultani_py3/AnomalyDetectionCVPR2018-master/Features/Abnormal\Explosion045_x